In [4]:
import pandas as pd
import numpy as np
import joblib
import json
import os
import warnings
import plotly.express as px
import plotly.graph_objects as go
import onnxruntime as ort
from sklearn.metrics import (
    accuracy_score, f1_score, recall_score, 
    confusion_matrix, roc_curve, auc, 
    precision_recall_curve, average_precision_score
)
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance

warnings.filterwarnings('ignore')

# 분석에 필요한 파일 경로와 결과 이미지를 저장할 위치를 설정합니다
metrics_path = '../data/model_metrics.json'
ml_model_path = '../models/spotify_churn_model.pkl'
dl_model_path = '../models/spotify_dl_model.onnx'
dl_preprocessor_path = '../models/dl_preprocessor.pkl'
data_path = '../data/spotify_churn_dataset.csv'
image_dir = '../02_training_report/images/'

if not os.path.exists(image_dir):
    os.makedirs(image_dir)

# 모델별 성능 지표를 불러와 데이터프레임으로 정리합니다
with open(metrics_path, 'r', encoding='utf-8') as f:
    metrics = json.load(f)

df_metrics = pd.DataFrame(metrics).T.reset_index().rename(columns={
    'index': '모델',
    'Accuracy': '정확도',
    'Recall': '재현율',
    'F1-Score': 'F1 스코어'
})

df_metrics['모델'] = df_metrics['모델'].replace({
    'Deep Learning (DNN)': '딥러닝(DNN)', 
    'RandomForest': '머신러닝(RF)',
    'Deep Learning (PyTorch_ONNX)': '딥러닝(DNN)'
})

# 1. 모델별 주요 성능 지표 시각화
fig_performance = px.bar(
    df_metrics, x='모델', y=['정확도', '재현율', 'F1 스코어'],
    barmode='group', text_auto='.4f', 
    title='모델별 주요 성능 지표 비교 (정확도, 재현율, F1)',
    labels={'value': '점수', 'variable': '지표'},
    color_discrete_sequence=px.colors.qualitative.Vivid
)
fig_performance.update_traces(textposition='outside')
fig_performance.update_layout(yaxis_range=[0, 1.2])
fig_performance.write_image(f'{image_dir}model_performance_bar.png')
fig_performance.show()

# 2. 데이터 준비 및 모델 로드
df = pd.read_csv(data_path)
if 'user_id' in df.columns: df = df.drop(columns=['user_id'])
df['ad_burden'] = df['ads_listened_per_week'] / (df['listening_time'] + 1)
df['satisfaction_score'] = df['songs_played_per_day'] * (1 - df['skip_rate'])
df['time_per_song'] = df['listening_time'] / (df['songs_played_per_day'] + 1)

X = df.drop(columns=['is_churned'])
y = df['is_churned']
_, X_test, _, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

ml_model = joblib.load(ml_model_path)
dl_prep = joblib.load(dl_preprocessor_path)
ort_session = ort.InferenceSession(dl_model_path)

# 3. 사이킷런 호환 딥러닝 래퍼 클래스
class DLWrapper:
    def __init__(self, session, prep, threshold):
        self.session = session
        self.prep = prep
        self.threshold = threshold
        self._estimator_type = "classifier"
        self.classes_ = np.array([0, 1])
    
    def fit(self, X, y): return self
    
    def predict(self, X):
        X_proc = self.prep.transform(X).astype(np.float32)
        input_name = self.session.get_inputs()[0].name
        probs = self.session.run(None, {input_name: X_proc})[0].flatten()
        return (probs >= self.threshold).astype(int)

# 4. 특성 중요도 계산 및 시각화 (수치 실종 및 겹침 문제 해결)
print("특성 중요도를 산출하는 중입니다. 잠시만 기다려 주세요...")
dl_threshold = metrics.get('Deep Learning (DNN)', {}).get('Best Threshold', 0.5)
dl_model_wrapped = DLWrapper(ort_session, dl_prep, dl_threshold)

r_ml = permutation_importance(ml_model, X_test, y_test, n_repeats=5, random_state=42, scoring='f1')
r_dl = permutation_importance(dl_model_wrapped, X_test, y_test, n_repeats=5, random_state=42, scoring='f1')

def plot_importance(importance_results, title, file_name, color):
    df_imp = pd.DataFrame({'특성': X_test.columns, '중요도': importance_results.importances_mean})
    total = df_imp['중요도'].abs().sum()
    df_imp['기여도_pct'] = (df_imp['중요도'] / total) * 100
    df_imp = df_imp.sort_values(by='기여도_pct', ascending=True)

    fig = px.bar(
        df_imp, x='기여도_pct', y='특성', orientation='h', 
        title=title, color_discrete_sequence=[color],
        labels={'기여도_pct': '기여도 (%)'}
    )
    
    # 수치 앞에 공백을 추가하여 특성 이름과 겹치지 않게 하고, 강제로 표시하도록 설정합니다
    fig.update_traces(
        texttemplate=' %{x:.2f}%', 
        textposition='outside',
        cliponaxis=False
    )
    fig.update_layout(
        margin=dict(l=200, r=80, t=80, b=50), # 왼쪽 여백을 충분히 주어 겹침 방지
        uniformtext_minsize=10, # 텍스트 크기 최소값 설정
        uniformtext_mode='show', # 아주 작은 값도 숨기지 않고 강제로 표시
        height=600,
        xaxis=dict(range=[0, df_imp['기여도_pct'].max() * 1.25]) # 수치 공간 확보를 위해 범위 확장
    )
    fig.write_image(f'{image_dir}{file_name}')
    fig.show()

plot_importance(r_ml, '머신러닝(RF) 특성 중요도 분석 (%)', 'ml_importance.png', '#1DB954')
plot_importance(r_dl, '딥러닝(DNN) 특성 중요도 분석 (%)', 'dl_importance.png', '#3498DB')

# 5. 혼동 행렬 시각화
y_prob_ml = ml_model.predict_proba(X_test)[:, 1]
ml_thresh = metrics.get('RandomForest', {}).get('Best Threshold', 0.5)
y_pred_ml = (y_prob_ml >= ml_thresh).astype(int)
y_pred_dl = dl_model_wrapped.predict(X_test)

for y_pred, name, file, col in zip([y_pred_ml, y_pred_dl], ['머신러닝(RF)', '딥러닝(DNN)'], ['ml_cm.png', 'dl_cm.png'], ['Greens', 'Blues']):
    cm = confusion_matrix(y_test, y_pred)
    fig_cm = px.imshow(cm, text_auto=True, color_continuous_scale=col, x=['유지', '이탈'], y=['유지', '이탈'], title=f'{name} 혼동 행렬')
    fig_cm.update_layout(xaxis_title='예측된 상태', yaxis_title='실제 상태')
    fig_cm.write_image(f'{image_dir}{file}')
    fig_cm.show()

# 6. ROC 및 PR 커브 분석
X_test_dl_proc = dl_prep.transform(X_test).astype(np.float32)
y_prob_dl = ort_session.run(None, {ort_session.get_inputs()[0].name: X_test_dl_proc})[0].flatten()

for prob, label, roc_file, pr_file, color in zip(
    [y_prob_ml, y_prob_dl], ['머신러닝(RF)', '딥러닝(DNN)'], 
    ['ml_roc.png', 'dl_roc.png'], ['ml_pr.png', 'dl_pr.png'], ['#1DB954', '#3498DB']
):
    fpr, tpr, _ = roc_curve(y_test, prob)
    roc_score = auc(fpr, tpr)
    fig_roc = go.Figure()
    fig_roc.add_trace(go.Scatter(x=fpr, y=tpr, name=f'AUC = {roc_score:.4f}', line=dict(color=color, width=3)))
    fig_roc.add_shape(type='line', line=dict(dash='dash'), x0=0, x1=1, y0=0, y1=1)
    fig_roc.update_layout(title=f'{label} ROC 커브 (AUC: {roc_score:.4f})', xaxis_title='FPR', yaxis_title='TPR')
    fig_roc.write_image(f'{image_dir}{roc_file}')
    
    prec, rec, _ = precision_recall_curve(y_test, prob)
    pr_score = average_precision_score(y_test, prob)
    fig_pr = go.Figure()
    fig_pr.add_trace(go.Scatter(x=rec, y=prec, name=f'PR-AUC = {pr_score:.4f}', line=dict(color=color, width=3)))
    fig_pr.update_layout(title=f'{label} 정밀도-재현율 커브 (점수: {pr_score:.4f})', xaxis_title='Recall', yaxis_title='Precision')
    fig_pr.write_image(f'{image_dir}{pr_file}')

print("✅ 시각화 수치가 사라지거나 겹치는 문제를 해결하여 저장을 완료했습니다.")

특성 중요도를 산출하는 중입니다. 잠시만 기다려 주세요...


✅ 시각화 수치가 사라지거나 겹치는 문제를 해결하여 저장을 완료했습니다.
